In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from msedge.selenium_tools import Edge, EdgeOptions
from selenium import webdriver
from requests_html import HTMLSession

session = HTMLSession()
driver = webdriver.Chrome("C:/Users/1/Desktop/MIPT_education/DataTon-2022/chromedriver.exe")

def extract_items(item):
    try:
        atag = item.h2.a
        description = (atag.text.strip())
        url  = 'https://www.amazon.com' + atag.get('href')
    except: 
        return None
    
    try:
        price_parent = item.find('span','a-price-whole').text
    except: return None
    try:
        stars = item.i.text
        reviews_num = item.find('span',{'class':'a-size-base s-underline-text'}).text
    except AttributeError:
        stars=''
        reviews_num=''
    result = (description,url,price_parent,stars,reviews_num)
    return result

records = []
url = 'https://www.amazon.com/s?k=laptop&crid=2JUKYBAPLWFUW&page={}'
for page in range(1,21):
    driver.get(url.format(page))
    soup = bs(driver.page_source, 'html.parser')
    results = soup.find_all('div',{'class':'a-section a-spacing-small a-spacing-top-small'})
    for item in results:
        record = extract_items(item)
        if record:
            records.append(record)
driver.close()

df_res=pd.DataFrame()
records_to_df_tuple = tuple(records_to_df)
df_res = pd.DataFrame(records_to_df_tuple,columns=['description','url','price','stars','reviews_num'])
df_res.to_csv('Amazon_20_pages_laptops.csv')